In [ ]:
# Import der benötigten Bibliotheken
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

Vorbereitung der Daten


In [ ]:
# Daten-Vorbereitung
base_dir = '/Users/mukunthanpremraj/Downloads/Mandatory Sauber/Datens則ze/pokemon'  # Pfad zum übergeordneten Ordner, der die Klassenordner enthält
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Behalte die Aufteilung für Training und Validation bei
)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Spezifiziert, dass dies der Trainingssatz ist
)

validation_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Spezifiziert, dass dies der Validierungssatz ist
)

Modell Setup

In [ ]:
# Modell-Setup
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Zuerst wird das Basismodell eingefroren

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Dropout hinzugefügt, um Overfitting zu reduzieren
predictions = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

Training des Models

In [ ]:
# Trainieren des Modells
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=20,
    callbacks=[
        ModelCheckpoint('best_model.keras', save_best_only=True),
        EarlyStopping(monitor='val_loss', patience=5),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2)
    ]
)

Fine Tuning des Modells

In [ ]:
# Fine-Tuning des Modells
for layer in base_model.layers:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

Bewertung und Ergebnisse

In [ ]:
# Ergebnisse bewerten
eval_result = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print(f'Performance vor dem Fine-Tuning: {history.history["val_accuracy"][-1]}')
print(f'Performance nach dem Fine-Tuning: {history_fine.history["val_accuracy"][-1]}')

In [ ]:
model.save('mein_modell.h5') 